In [1]:
import pickle
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical 

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

D:\Anaconda\Conda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
os.chdir('D:\EEG to emotion\EEG_DE_features')

In [3]:
train=pd.read_csv("seed_2.csv")
Y=pd.DataFrame({'label':train['label']},index=None)

In [4]:
train.drop(['Unnamed: 0','label'],axis=1,inplace=True)

In [5]:
train['combine'] = train.values.tolist()

In [6]:
for i in range(train['combine'].shape[0]):
    train['combine'][i]=np.asarray(train['combine'][i]) ##merge the data to form an array

C:\Users\admin\AppData\Local\Temp/ipykernel_6324/1822833573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['combine'][i]=np.asarray(train['combine'][i]) ##merge the data to form an array


In [7]:
data = []
label = []
for i in range(0,train.shape[0]):
    data.append(train['combine'][i])
    label.append(Y['label'][i])

In [8]:
new_train=np.asarray(data)
new_train=new_train.reshape(new_train.shape[0],new_train.shape[1],1)

In [9]:
label_new=np.asarray(label)

In [10]:
label_new = to_categorical(label_new)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(new_train, label_new, test_size=0.1, random_state=42)

In [12]:
model=load_model('1Dcnn.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 310, 128)          512       
                                                                 
 batch_normalization (BatchN  (None, 310, 128)         512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 155, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 155, 128)          49280     
                                                                 
 batch_normalization_1 (Batc  (None, 155, 128)         512       
 hNormalization)                                                 
                                                        

In [14]:
df=pd.read_csv('Books_100k.csv')

In [15]:
y_pred=model.predict(x_test)

In [16]:
y=np.zeros((y_pred.shape[0],5))
for i in range(y_pred.shape[0]):
    y[i]=tf.math.sigmoid(y_pred[i])

In [17]:
emotion2genre_matrix=np.array(
[[10,13,29,6,17,21,28,6,10,28,9,15,8,11,10,17,8,16,14,23,23],                                  #disgust
 [6,12,54,4,6,28,30,8,8,52,11,16,9,14,11,15,11,38,13,14,14],                                   #fear
 [8,10,35,4,11,21,23,7,8,38,10,13,8,23,20,23,9,29,12,15,21],                                   #sad
 [34,25,35,9,40,40,50,24,27,33,13,37,18,19,13,22,10,40,40,51,56],                              #neutral
 [24,16,33,5,30,38,56,19,17,34,13,44,11,12,13,17,6,45,41,52,57]]                               #happy 
 )

In [18]:
emotion2genre_matrix = normalize(emotion2genre_matrix, axis=1, norm='l2')

In [19]:
y_pred=np.matmul(y,emotion2genre_matrix)

In [20]:
y_pred=normalize(y_pred, axis=1, norm='l2')

In [21]:
y_1=y_pred[0]

In [22]:
ones = np.ones((1055, 21))

In [23]:
y_1=y_1*ones

In [24]:
genres=["Business", "Classics", "Comics", "Contemporary", "Crime", "Fantasy", "Fiction",
        "History", "Horror", "Humor", "Manga", "Mystery", "Nonfiction","Philosophy", "Poetry",
        "Psychology", "Religion", "Romance", "Science", "Suspense","Thriller"]

In [25]:
vectors = df[genres].values

In [26]:
normalized_vectors = normalize(vectors, axis=1, norm='l2')

In [27]:
similarity = cosine_similarity(y_1, normalized_vectors)

In [28]:
df['similarity'] = similarity[0]

In [29]:
books= df.sort_values(by=['similarity', 'rating'], ascending=False) \
            .head(20)['title'] \
            .sample(frac=0.5)

In [30]:
book_1=books.iloc[0]
book_2=books.iloc[1]
book_3=books.iloc[2]
book_4=books.iloc[3]
book_5=books.iloc[4]

In [31]:
print(book_1)
print(book_2)
print(book_3)
print(book_4)
print(book_5)

Swamp Thing, Vol. 3: The Curse
The New Teen Titans Archives, Vol. 1
Green Arrow, Vol. 2: Sounds of Violence
Kebakaran Hutan
The Complete Peanuts, Vol. 15: 1979-1980
